# Realign two EEGLab.Epoch object's epochs.

@Author [@FranckPrts](https://github.com/FranckPrts).

Here we attempt to provide a python-based solution to the following question: **How to can I realign concurrent epochs from two epoched object?**

Our main goal here is to be able to target all epochs in a given EEG that doesn't have a *sister* (concurently recorded) epoch in the other EEG because it was rejected by preprocessing. At the end, we should have two epoched EEG with the same amount of epochs and where each pair of epoch has their *sister* epoch at the same index.

Fist, the EEG data of each participant was segmented in 1sec epochs before moving on to preprocessing independelty each two EEG data. For each stage of this iterative process (in our case 2-3 iterations) the ID of the rejected epochs were noted in a separate file.

Our issue arise from the fact that once each step was performed, saving the data would lead to losing track of what was the epochs original IDs. 
As exemplified below: 

#IMAGE

In that process we see that an epoch that originally had the ID #6 can end up with the new ID #3. 

To retrieve the original id of the epoch, we will have to work bakward from the last iteration of preprocessing to the first iteration. At each step we will store what was the previous ID of the epochs so we can find their original IDs. 

## Imports

In [1]:
# Package 
import mne
import numpy as np
import pandas as pd

# Custom functions
import utils

We import two eeg stream that were preprocessed in MATLAB

In [28]:
dy

array(['220', ' FINS_220_Child_FreePlay_xchan_rej3.set',
       ' FINS_220_Adult_FreePlay_xchan_ica_rej3.set'], dtype='<U65')

In [30]:
files_to_process = np.loadtxt("files_to_process.csv",
                 delimiter=",", dtype=str)

dyad = [x for x in files_to_process]
# Careful, the file_to_process is in the order (dyad_nb, eeg_filepath_child, eeg_filepath_adutl)
dy = dyad[0]
data_path = '../FINS-data/'

eeg1 = mne.io.read_epochs_eeglab('{}{}_{}_FP/{}'.format(data_path, dy[0], 'child', dy[1])) 
eeg2 = mne.io.read_epochs_eeglab('{}{}_{}_FP/{}'.format(data_path, dy[0], 'adult', dy[2]))

Extracting parameters from /Users/zoubou/Documents/Work/NYU/Brito-Lab/FINS-Codes/../FINS-data/220_child_FP/FINS_220_Child_FreePlay_xchan_rej3.set...
Not setting metadata
159 matching events found
No baseline correction applied
0 projection items activated
Ready.
Extracting parameters from /Users/zoubou/Documents/Work/NYU/Brito-Lab/FINS-Codes/../FINS-data/220_adult_FP/FINS_220_Adult_FreePlay_xchan_ica_rej3.set...
Not setting metadata
206 matching events found
No baseline correction applied
0 projection items activated
Ready.


/var/folders/vv/stc9rswn5c95vxdzpx7z6qqr0000gn/T/ipykernel_66189/3894970619.py:9: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  eeg1 = mne.io.read_epochs_eeglab('{}{}_{}_FP/{}'.format(data_path, dy[0], 'child', dy[1]))
/var/folders/vv/stc9rswn5c95vxdzpx7z6qqr0000gn/T/ipykernel_66189/3894970619.py:10: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  eeg2 = mne.io.read_epochs_eeglab('{}{}_{}_FP/{}'.format(data_path, dy[0], 'adult', dy[2]))


Let's see how many epochs we have per EEG file:

In [35]:
type(eeg1)

mne.io.eeglab.eeglab.EpochsEEGLAB

## Useful references

- To get comfortable with the MNE documentation, you should know that MNE is based on python [Object Oriented Programming (00P)](https://realpython.com/python3-object-oriented-programming/). These objects are defined from a python `Class`.
    - You can get familiarized with the OOP structure and its componenent, e.g. `methods` (a function associated to the the object) and `attribute` (a variable associated to the object), wit [this tutorial](https://www.datacamp.com/tutorial/python-oop-tutorial).
    - In MNE, we find [`Raw` objects](https://mne.tools/stable/generated/mne.io.Raw.html) (continuous data) or [`Epoch` objects](https://mne.tools/stable/generated/mne.Epochs.html) (a collection of epochs). 

You can find an introduction to the **Epochs data structure** [here](https://mne.tools/stable/auto_tutorials/epochs/10_epochs_overview.html) in MNE. 